In [1]:
import pandas as pd 

In [3]:
# Load your dataset
df = pd.read_csv('/home/pmen/Shortcuts-Problem-in-Medical-Image-Analysis/shortcuts-chest-xray/subsets/30k-2-val.csv')

# Replace 'CheXpert-v1.0' with 'CheXpert-v1.0-small' in the 'Path' column
df['Path'] = df['Path'].str.replace('CheXpert-v1.0', 'CheXpert-v1.0-small')

# Optionally, save the modified DataFrame back to a CSV file
df.to_csv('/home/pmen/Shortcuts-Problem-in-Medical-Image-Analysis/shortcuts-chest-xray/subsets2/30k-2-val.csv', index=False)

<ipython-input-3-093c30e57e19>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Path'] = df['Path'].str.replace('CheXpert-v1.0', 'CheXpert-v1.0-small')


In [1]:

import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")


Using GPU: NVIDIA GeForce GTX 1080 Ti


In [17]:
import numpy as np
from torch.utils.data import Dataset
import cv2
import os
from PIL import Image
from data.imgaug import GetTransforms
from data.utils import transform

np.random.seed(0)


class ImageDataset(Dataset):
    def __init__(self, label_path, cfg, mode='train'):
        self.cfg = cfg
        self._label_header = None
        self._image_paths = []
        self._labels = []
        self._senstive_attribute = []
        self._mode = mode
        self.dict = {'1.0': '1', '1': '1', '': '0', '0.0': '0', '0': '0', '-1.0': '0', '-1': '0', 'Female': '1', 'Male': '0', 'Unknown': '0'}
        with open(label_path) as f:
            header = f.readline().strip('\n').split(',')
            idx_pneumothorax = header.index('Pneumothorax')
            idx_sensitive_attribute = header.index(cfg.get('sensitive_attribute'))
            self._label_header = [header[idx_pneumothorax]]
            for line in f:
                fields = line.strip('\n').split(',')

                labels = [int(self.dict.get(fields[idx_pneumothorax]))]
                self._labels.append(labels)

                sensitive_attribute = [int(self.dict.get(fields[idx_sensitive_attribute]))]
                self._senstive_attribute.append(sensitive_attribute)

                image_path = fields[0]
                image_path = os.path.join(cfg.base_path, os.path.join(*(image_path.split(os.path.sep)[1:])))
                self._image_paths.append(image_path)
                assert os.path.exists(image_path), image_path

                break

        self._num_image = len(self._image_paths)

    def __len__(self):
        return self._num_image

    def __getitem__(self, idx):
        image = cv2.imread(self._image_paths[idx], 0)
        image = Image.fromarray(image)
        if self._mode == 'train':
            image = GetTransforms(image, type=self.cfg.use_transforms_type)
        image = np.array(image)
        image = transform(image, self.cfg)
        labels = np.array(self._labels[idx]).astype(np.float32)
        sensitive_attribute = np.array(self._senstive_attribute[idx]).astype(np.float32)

        path = self._image_paths[idx]

        if self._mode == 'train' or self._mode == 'dev':
            return (image, labels, sensitive_attribute)
        elif self._mode == 'test':
            return (image, path)
        elif self._mode == 'heatmap':
            return (image, path, labels)
        else:
            raise Exception('Unknown mode : {}'.format(self._mode))


In [22]:
from torch.utils.data import DataLoader
dataloader_train = DataLoader(
        ImageDataset(cfg.train_csv, cfg, mode='train'),
        batch_size=1, num_workers=0,
        drop_last=True, shuffle=True)

In [23]:
dataloader_train

In [18]:
data =  ImageDataset(cfg.train_csv, cfg, mode='train')

In [20]:
data.__getitem__(0)

(array([[[0.109375, 0.109375, 0.109375, ..., 0.109375, 0.109375,
          0.109375],
         [0.109375, 0.109375, 0.109375, ..., 0.109375, 0.109375,
          0.109375],
         [0.109375, 0.109375, 0.109375, ..., 0.109375, 0.109375,
          0.109375],
         ...,
         [0.      , 0.      , 0.      , ..., 0.      , 0.      ,
          0.      ],
         [0.      , 0.      , 0.      , ..., 0.      , 0.      ,
          0.      ],
         [0.      , 0.      , 0.      , ..., 0.      , 0.      ,
          0.      ]],
 
        [[0.109375, 0.109375, 0.109375, ..., 0.109375, 0.109375,
          0.109375],
         [0.109375, 0.109375, 0.109375, ..., 0.109375, 0.109375,
          0.109375],
         [0.109375, 0.109375, 0.109375, ..., 0.109375, 0.109375,
          0.109375],
         ...,
         [0.      , 0.      , 0.      , ..., 0.      , 0.      ,
          0.      ],
         [0.      , 0.      , 0.      , ..., 0.      , 0.      ,
          0.      ],
         [0.      , 0. 

In [12]:
print(cfg.get('sensitive_attribute'))  # This should not print None if everything is correct
# 

Sex


In [11]:
config_path = '/home/pmen/Shortcuts-Problem-in-Medical-Image-Analysis/shortcuts-chest-xray/config/config.json'
from ml_collections import ConfigDict
# Assuming you have loaded your config as a dictionary named cfg
with open(config_path, 'r') as f:
    cfg = json.load(f)

# Convert the dictionary to a ConfigDict which allows attribute-like access
cfg = ConfigDict(cfg)

In [1]:
import os

new_directory = "/home/pmen/Shortcuts-Problem-in-Medical-Image-Analysis/shortcuts-chest-xray"

os.chdir(new_directory)
